In [1]:
!pip install pyspark

In [2]:
!pip install spark

In [3]:
import pandas as pd
import pickle
import numpy as np
from google.colab import drive
from datetime import datetime, timedelta
drive.mount('/content/drive')
pickle_path = '/content/drive/MyDrive/trendyol_bootcamp_capstone/'

import pickle

import pandas as pd
import numpy as np
import random
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn import model_selection

import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def load_pickle(filename):
  infile = open(pickle_path+filename,'rb')
  loaded_data = pickle.load(infile)
  infile.close()
  return loaded_data

In [5]:
train_data = load_pickle('train_data.pickle')
train_data.reset_index(drop=True,inplace=True)

In [6]:
train_data['timestamp'] = (train_data['t_dat'] - train_data['t_dat'].iloc[0])/pd.to_timedelta('1Min')

In [7]:
train_data.drop(columns = ['t_dat','price','sales_channel_id'], inplace = True)

In [8]:
train_data['rank'] = 1

In [9]:
train_data = train_data
train_data

,customer_id,article_id,timestamp,rank
0,00075ef36696a7b4ed8c83e22a4bf7ea7c90ee110991ec...,856440001,0.0,1
1,00075ef36696a7b4ed8c83e22a4bf7ea7c90ee110991ec...,756132001,0.0,1
2,00075ef36696a7b4ed8c83e22a4bf7ea7c90ee110991ec...,873915001,0.0,1
3,0007c3ba357b0e54131e54d7a5d619263c5a9e6826308b...,841228001,0.0,1
4,0007c3ba357b0e54131e54d7a5d619263c5a9e6826308b...,832361003,0.0,1
...,...,...,...,...
7690355,fff2282977442e327b45d8c89afde25617d00124d0f999...,919786002,244800.0,1
7690356,fffbdd2f8e59d45c0fb50a14b0ea555f7daa5307b3f922...,765308002,244800.0,1
7690357,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,244800.0,1
7690358,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,884081001,244800.0,1


In [10]:
customer_id_dict = {}
for i,id in enumerate(list(set(train_data.customer_id))):
  customer_id_dict[id] = i
"""
train_data[:1000000].replace({"customer_id": customer_id_dict},inplace = True)
train_data[1000000:2000000].replace({"customer_id": customer_id_dict},inplace = True)
train_data[2000000:4000000].replace({"customer_id": customer_id_dict},inplace = True)
train_data[4000000:6000000].replace({"customer_id": customer_id_dict},inplace = True)

for i in range (0,len(train_data),40000):
  print("** " + str(i))
  train_data[i:i+40000].replace({"customer_id": customer_id_dict},inplace = True)
"""

'\ntrain_data[:1000000].replace({"customer_id": customer_id_dict},inplace = True)\ntrain_data[1000000:2000000].replace({"customer_id": customer_id_dict},inplace = True)\ntrain_data[2000000:4000000].replace({"customer_id": customer_id_dict},inplace = True)\ntrain_data[4000000:6000000].replace({"customer_id": customer_id_dict},inplace = True)\n\nfor i in range (0,len(train_data),40000):\n  print("** " + str(i))\n  train_data[i:i+40000].replace({"customer_id": customer_id_dict},inplace = True)\n'

In [11]:
mapper = pd.DataFrame.from_dict(customer_id_dict, orient='index')
mapper = mapper.reset_index(drop=False).rename(columns={'index':'customer_id', 0:'customer_int'})
train_data = train_data.merge(mapper, on='customer_id')
train_data['customer_id'] = train_data['customer_int']
train_data.drop(columns = ['customer_int'])

,customer_id,article_id,timestamp,rank
0,134168,856440001,0.0,1
1,134168,756132001,0.0,1
2,134168,873915001,0.0,1
3,134168,777756002,12960.0,1
4,134168,861565001,12960.0,1
...,...,...,...,...
7690355,289580,842028004,244800.0,1
7690356,289580,881577001,244800.0,1
7690357,289580,923340001,244800.0,1
7690358,119217,881942001,244800.0,1


In [18]:
train_data['timestamp'] = train_data['timestamp'].astype(int)

In [20]:
train_data.drop(columns = ['customer_int'], inplace = True)

In [21]:
train_data

,customer_id,article_id,timestamp,rank
0,134168,856440001,0,1
1,134168,756132001,0,1
2,134168,873915001,0,1
3,134168,777756002,12960,1
4,134168,861565001,12960,1
...,...,...,...,...
7690355,289580,842028004,244800,1
7690356,289580,881577001,244800,1
7690357,289580,923340001,244800,1
7690358,119217,881942001,244800,1


In [22]:
train_data.to_csv("train_data.txt", header = False, sep = ",", index = False)

In [ ]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

from pyspark.sql import SparkSession

# $example on$
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
# $example off$

if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("ALSExample")\
        .getOrCreate()

    # $example on$
    lines = spark.read.text("train_data.txt").rdd
    parts = lines.map(lambda row: row.value.split(","))
    ratingsRDD = parts.map(lambda p: Row(user_id=int(p[0]), item_id=int(p[1]),
                                         rating=float(p[3]), timestamp=int(p[2])))
    ratings = spark.createDataFrame(ratingsRDD)
    (training, test) = ratings.randomSplit([0.8, 0.2])

    # Build the recommendation model using ALS on the training data
    # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
    als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
              coldStartStrategy="drop", implicitPrefs=False)
    model = als.fit(training)

    # Evaluate the model by computing the RMSE on the test data
    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))

    # Generate top 10 movie recommendations for each user
    userRecs = model.recommendForAllUsers(10)
    # Generate top 10 user recommendations for each movie
    movieRecs = model.recommendForAllItems(10)

    # Generate top 10 movie recommendations for a specified set of users
    users = ratings.select(als.getUserCol()).distinct().limit(3)
    userSubsetRecs = model.recommendForUserSubset(users, 10)
    # Generate top 10 user recommendations for a specified set of movies
    movies = ratings.select(als.getItemCol()).distinct().limit(3)
    movieSubSetRecs = model.recommendForItemSubset(movies, 10)
    # $example off$
    userRecs.show()
    movieRecs.show()
    userSubsetRecs.show()
    movieSubSetRecs.show()

    spark.stop()

Root-mean-square error = 0.09947698546371402


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning
